# JPL DE430 and PlanetaryEphemeris.jl

In [ ]:
versioninfo()

In [ ]:
;pwd

In [ ]:
]st

In [ ]:
using Revise

In [ ]:
using PlanetaryEphemeris

In [ ]:
using TaylorIntegration
using Dates, LinearAlgebra

In [ ]:
using SPICE

In [ ]:
furnsh("/Users/Jorge/projects/jpleph/de430_1850-2150.bsp")

In [ ]:
furnsh("/Users/Jorge/projects/jpleph/naif0012.tls")

In [ ]:
furnsh("/Users/Jorge/projects/jpleph/ast343de430.bsp")

In [ ]:
furnsh("/Users/Jorge/projects/jpleph/TTmTDB.de430.19feb2015.bsp")

In [ ]:
using Statistics

In [ ]:
using JLD2

In [ ]:
#eulang_de430 = load("../eph/moon_pa_de430_2000_2036_et.jld", "eulang_de430");
#eulang_de430 = load("/Users/Jorge/.julia/dev/JPLEphemeris/moon_pa_de430_1969_2069_et.jld", "eulang_de430");

## Load ephemeris from file

In [ ]:
#ss16ast_eph_fwd_fname = "sseph343ast016_p6y_et.jld"
#ss16ast_eph_fwd_fname = "sseph343ast016_p31y_et_1969Jun28.jld"
#ss16ast_eph_fwd_fname = "sseph343ast016_p30y_et_J2000.jld"
ss16ast_eph_fwd_fname = "sseph343ast016_p100y_et.jld2"
@time ss16ast_eph_fwd = load(ss16ast_eph_fwd_fname, "ss16ast_eph");

In [ ]:
day0 = ss16ast_eph_fwd.t0

In [ ]:
jd0 = J2000 + et0

In [ ]:
N = Int((size(ss16ast_eph_fwd.x)[2]-13)/6) ### <<<--- should use this with new version

In [ ]:
ss16ast_eph_fwd.t0

In [ ]:
findfirst(x->x>0.0, ss16ast_eph_fwd.t0 .+ ss16ast_eph_fwd.t)

In [ ]:
length(ss16ast_eph_fwd.t)

In [ ]:
size(ss16ast_eph_fwd.x)

In [ ]:
length(ss16ast_eph_fwd.t[4642:end])

In [ ]:
size(ss16ast_eph_fwd.x[4642:end,:])

In [ ]:
#ss16ast_2000_t0 = ss16ast_eph_fwd.t0
#ss16ast_2000_t = ss16ast_eph_fwd.t[4642:end]
#ss16ast_2000_x = ss16ast_eph_fwd.x[4642:end,:]
#ss16ast_2000 = TaylorInterpolant(ss16ast_2000_t0, ss16ast_2000_t, ss16ast_2000_x);
#jldopen("sseph343ast016_p70y_et_2000_2069.jld", "w") do file
#    addrequire(file, TaylorSeries)
#    addrequire(file, PlanetaryEphemeris)
#    write(file, "ss16ast_eph", ss16ast_2000)
#end

## Alternatively, compute ephemeris here

## Process data

In [ ]:
ss16ast_eph_fwd.t0, ss16ast_eph_fwd.t[end]/yr

In [ ]:
ss16ast_eph_fwd.t[end]

In [ ]:
tv_fwd = ss16ast_eph_fwd.t0 .+ ss16ast_eph_fwd.t;

In [ ]:
ss16ast_eph_fwd(tv_fwd[1])[end], spkgeo(1000000001, tv_fwd[1]*daysec, "J2000", 1000000000)[1][1]

In [ ]:
dttmtdb[50]

In [ ]:
tv_fwd[1], tv_fwd[end]

In [ ]:
julian2datetime(J2000 + tv_fwd[end])

In [ ]:
function eph_xv_et(naifid::Int, et::Float64, observer_id::Int=10)
    pv = spkgeo(naifid, et, "J2000", observer_id)[1]
    pv /= au
    pv[4:6] *= daysec
    return pv
end

In [ ]:
#ephX = ss16ast_eph_bwd_fwd
ephX = ss16ast_eph_fwd
;

In [ ]:
tvX = tv_fwd
tvX_yrs = 2000 .+ tvX/yr; # years (CE) # years since initial time

In [ ]:
yr0 = 2000 .+ day0/yr;

In [ ]:
# distribution of time-steps
d_tvX_days = diff(tvX[2:end-1])
mean(d_tvX_days), std(d_tvX_days)

In [ ]:
findmin(d_tvX_days[2:end-1])

In [ ]:
findmax(d_tvX_days)

In [ ]:
length(tv_fwd), length(tvX) # length(tv_bwd)

In [ ]:
issorted(tvX), issorted(tvX_yrs)

In [ ]:
dxsu = Array{Float64}(undef, length(tvX), 6)
dxme = Array{Float64}(undef, length(tvX), 6)
dxve = Array{Float64}(undef, length(tvX), 6)
dxea = Array{Float64}(undef, length(tvX), 6)
dxmo = Array{Float64}(undef, length(tvX), 6)
dxemb = Array{Float64}(undef, length(tvX), 6)
xmo_pleph = Array{Float64}(undef, length(tvX), 6)
xea_pleph = Array{Float64}(undef, length(tvX), 6)
xmo_de430 = Array{Float64}(undef, length(tvX), 6)
xea_de430 = Array{Float64}(undef, length(tvX), 6)
dxma = Array{Float64}(undef, length(tvX), 6)
dxju = Array{Float64}(undef, length(tvX), 6)
dxsa = Array{Float64}(undef, length(tvX), 6)
dxne = Array{Float64}(undef, length(tvX), 6)
dxce = Array{Float64}(undef, length(tvX), 6)
dxpl = Array{Float64}(undef, length(tvX), 6)
dxsy = Array{Float64}(undef, length(tvX), 6)

xEMB_pleph = Array{Float64}(undef, length(tvX), 6)
xME_pleph = Array{Float64}(undef, length(tvX), 6)
xEMB_de430 = Array{Float64}(undef, length(tvX), 6)
xME_de430 = Array{Float64}(undef, length(tvX), 6)

ttmtdb_pleph = Array{Float64}(undef, length(tvX), 1)
ttmtdb_de430 = Array{Float64}(undef, length(tvX), 1)
dttmtdb = Array{Float64}(undef, length(tvX), 1)

#eulang_pleph = Array{Float64}(undef, length(tvX), 3)
#eulang_de430_arr = Array{Float64}(undef, length(tvX), 3)
#deulang = Array{Float64}(undef, length(tvX), 3)

for i in eachindex(tvX)
    ssti = ephX(tvX[i])
    
    xvXS = ssti[nbodyind(N, 1)]
    dxsu[i,:] = (eph_xv_et( 10, tvX[i]*daysec,0) .- xvXS)*(1e3au)
    dxme[i,:] = (eph_xv_et(199, tvX[i]*daysec) .- (ssti[nbodyind(N, 2)]-xvXS))*(1e3au)
    dxve[i,:] = (eph_xv_et(299, tvX[i]*daysec) .- (ssti[nbodyind(N, 3)]-xvXS))*(1e3au)
    
    # xEMB_pleph[i,:] = (ssti[nbodyind(N, ea)]-xvXS)*(1e3au)
    # xME_pleph[i,:] = (ssti[nbodyind(N, mo)])*(1e3au)
    # xEMB_de430[i,:] = eph_xv_et(3, tvX[i])*(1e3au)
    # xME_de430[i,:] = eph_xv_et(301, tvX[i], 399)*(1e3au)
    # xmo_pleph[i,:] = xEMB_pleph[i,:] .+ (μ[ea]*xME_pleph[i,:])/(μ[ea]+μ[mo])
    # xea_pleph[i,:] = xEMB_pleph[i,:] .- (μ[mo]*xME_pleph[i,:])/(μ[ea]+μ[mo])
    # xmo_de430[i,:] = xEMB_de430[i,:] .+ (μ[ea]*xME_de430[i,:])/(μ[ea]+μ[mo])
    # xea_de430[i,:] = xEMB_de430[i,:] .- (μ[mo]*xME_de430[i,:])/(μ[ea]+μ[mo])
    
    xea_pleph[i,:] = (ssti[nbodyind(N,ea)]-xvXS)*(1e3au)
    xmo_pleph[i,:] = (ssti[nbodyind(N,mo)]-xvXS)*(1e3au)
    xea_de430[i,:] = eph_xv_et(399, tvX[i]*daysec)*(1e3au)
    xmo_de430[i,:] = eph_xv_et(301, tvX[i]*daysec)*(1e3au)
    xEMB_pleph[i,:] = ( (μ[mo].*xmo_pleph[i,:]) .+ (μ[ea].*xea_pleph[i,:]) )./(μ[ea]+μ[mo])
    xME_pleph[i,:]  = xmo_pleph[i,:] .- xea_pleph[i,:] 
    xEMB_de430[i,:] = ( (μ[mo].*xmo_de430[i,:]) .+ (μ[ea].*xea_de430[i,:]) )./(μ[ea]+μ[mo])
    xME_de430[i,:]  = xmo_de430[i,:] .- xea_de430[i,:]

    ttmtdb_de430[i] = eph_xv_et(1000000001, tvX[i]*daysec,1000000000)[1]*au
    ttmtdb_pleph[i] = ssti[end]
    # @show tvX[i], ttmtdb_de430[i], ttmtdb_pleph[i], ttmtdb_de430[i]-ttmtdb_pleph[i]
    
    dxea[i,:] = xea_de430[i,:] .- xea_pleph[i,:]
    dxmo[i,:] = xmo_de430[i,:] .- xmo_pleph[i,:]
    dxemb[i,:] = xEMB_de430[i,:] .- xEMB_pleph[i,:]

    dxma[i,:] = (eph_xv_et(      4, tvX[i]*daysec) .- (ssti[nbodyind(N, 6)]-xvXS))*(1e3au)
    dxju[i,:] = (eph_xv_et(      5, tvX[i]*daysec) .- (ssti[nbodyind(N, 7)]-xvXS))*(1e3au)
    dxsa[i,:] = (eph_xv_et(      6, tvX[i]*daysec) .- (ssti[nbodyind(N, 8)]-xvXS))*(1e3au)
    dxne[i,:] = (eph_xv_et(      8, tvX[i]*daysec) .- (ssti[nbodyind(N,10)]-xvXS))*(1e3au)
    dxpl[i,:] = (eph_xv_et(      9, tvX[i]*daysec) .- (ssti[nbodyind(N,11)]-xvXS))*(1e3au)
    dttmtdb[i] = ttmtdb_de430[i] - ttmtdb_pleph[i]
    #dxce[i,:] = (eph_xv_et(2000001, tvX[i]*daysec) .- (ssti[nbodyind(N,12)]-xvXS))*(1e3au)
    #dxsy[i,:] = (eph_xv_et(2000087, tvX[i]*daysec) .- (ssti[nbodyind(N,N)]-xvXS))*(1e3au)
    
    #eulang_pleph[i,:] = ssti[6N+1:6N+3]
    #eulang_de430_arr[i,:] = eulang_de430(tvX[i])
    #deulang[i,:] = eulang_de430_arr[i,:] .- eulang_pleph[i,:]
end

In [ ]:
# p100y (J2000), 0 ast
ind_ = lastindex(tvX);
1e3dxme[ind_,:]

In [ ]:
xlabel_str = "Year";

In [ ]:
@time using Plots

In [ ]:
gr()

In [ ]:
###Plots warmup
@time plot(rand(5), rand(5));

In [ ]:
plot( tvX_yrs, ttmtdb_de430, label="DE430")
plot!( tvX_yrs, ttmtdb_pleph, label="PE")
title!("TT-TDB: DE430 and PlEph.jl")
xlabel!(xlabel_str)
ylabel!("(TT-TDB) [seconds]")

In [ ]:
plot( tvX_yrs, ttmtdb_de430-ttmtdb_pleph, leg=false)
title!("Δ(TT-TDB)=(TT-TDB)(DE430) - (TT-TDB)(PE)")
#xlims!(2000,2001)
xlabel!(xlabel_str)
ylabel!("[seconds]")

In [ ]:
plot( tvX_yrs, 1e3dxsu[:,1], label="x")
plot!(tvX_yrs, 1e3dxsu[:,2], label="y")
plot!(tvX_yrs, 1e3dxsu[:,3], label="z")
title!("Sun barycentric position: DE430 - PlEph.jl (Float64)")
xlabel!(xlabel_str)
ylabel!("Delta (x,y,z) [mm]")

In [ ]:
plot( tvX_yrs, dxme[:,1], label="x")
plot!(tvX_yrs, dxme[:,2], label="y")
plot!(tvX_yrs, dxme[:,3], label="z")
title!("Mercury heliocentric position: DE430 - PlEph.jl (Float64)")
xlabel!(xlabel_str)
ylabel!("Delta (x,y,z) [m]")
#yticks!(-0.04:0.02:0.04)

In [ ]:
plot( tvX_yrs, dxve[:,1], label="x")
plot!(tvX_yrs, dxve[:,2], label="y")
plot!(tvX_yrs, dxve[:,3], label="z")
title!("Venus heliocentric position: DE430 - PlEph.jl (Float64)")
xlabel!(xlabel_str)
ylabel!("Delta (x,y,z) [m]")

In [ ]:
plot( tvX_yrs, dxemb[:,1], label="x")
plot!(tvX_yrs, dxemb[:,2], label="y")
plot!(tvX_yrs, dxemb[:,3], label="z")
title!("EMB heliocentric position: DE430 - PlEph.jl (Float64)")
xlabel!(xlabel_str)
ylabel!("Delta (x,y,z) [m]")
#savefig("emb_xyz.png")

In [ ]:
plot( tvX_yrs, dxea[:,1], label="x")
plot!(tvX_yrs, dxea[:,2], label="y")
plot!(tvX_yrs, dxea[:,3], label="z")
title!("Earth heliocentric position: DE430 - PlEph.jl (Float64)")
xlabel!(xlabel_str)
ylabel!("Delta (x,y,z) [m]")

In [ ]:
plot( tvX_yrs, dxmo[:,1], label="x", leg=:bottomleft)
plot!(tvX_yrs, dxmo[:,2], label="y")
plot!(tvX_yrs, dxmo[:,3], label="z")
title!("Moon heliocentric position: DE430 - PlEph.jl (Float64)")
xlabel!(xlabel_str)
ylabel!("Delta (x,y,z) [m]")
#yticks!(-7.5:2.5:7.5)
#savefig("moon_xyz.png")

In [ ]:
plot( tvX_yrs, xME_de430[:,1]-xME_pleph[:,1], label="x", leg=:bottomleft)
plot!(tvX_yrs, xME_de430[:,2]-xME_pleph[:,2], label="y")
plot!(tvX_yrs, xME_de430[:,3]-xME_pleph[:,3], label="z")
title!("Moon geocentric position: DE430 - PlEph.jl (Float64)")
xlabel!(xlabel_str)
ylabel!("Diff. in Moon geocentric position [m]")
ylabel!("Delta (x,y,z) [m]")
#xlims!(2000, 2006)
#ylims!(-10,10)
#savefig("moon_geocentric_xyz.png")


In [ ]:
dxem_pleph = (xmo_pleph .- xea_pleph)/(1e3au)
dxem_de430 = (xmo_de430 .- xea_de430)/(1e3au)
# a_M: Moon semimajor axis
a_M_pleph = semimajoraxis.(dxem_pleph[:,1], dxem_pleph[:,2], dxem_pleph[:,3], dxem_pleph[:,4], dxem_pleph[:,5], dxem_pleph[:,6], μ[ea], μ[mo])
a_M_de430 = semimajoraxis.(dxem_de430[:,1], dxem_de430[:,2], dxem_de430[:,3], dxem_de430[:,4], dxem_de430[:,5], dxem_de430[:,6], μ[ea], μ[mo]);
#diff_a_M = (1e3au)*(a_M_pleph.-a_M_de430);
diff_a_M = (1e3au)*(a_M_de430.-a_M_pleph);

In [ ]:
drem_pleph = (1e3au)*sqrt.(dxem_pleph[:,1].^2  + dxem_pleph[:,2].^2  + dxem_pleph[:,3].^2)
drem_de430 = (1e3au)*sqrt.(dxem_de430[:,1].^2  + dxem_de430[:,2].^2  + dxem_de430[:,3].^2);

In [ ]:
plot(tvX_yrs, 1e2(drem_de430-drem_pleph))
title!("Moon geocentric range: DE430 - PlEph.jl (Float64)")
ylabel!("d(E,M) [centimeters]")
xlabel!(xlabel_str)
#xlims!(-5,6)
#xticks!(-2000:1:2021)
#savefig("earth_moon_distance.png")

In [ ]:
plot(tvX_yrs, (1e3au)*a_M_pleph)
plot!(tvX_yrs, 1e3PlanetaryEphemeris.ld .+ 38.08e-3*tvX_yrs, label="LD + mean( aM_tidal_JPL(t) )")
title!("Moon semimajor axis: PlEph.jl (Float64)")
ylabel!("d(E,M) [centimeters]")
xlabel!(xlabel_str)

In [ ]:
plot( tvX_yrs, 1e3diff_a_M, label="aM(DE430)-aM(PlEph.jl)", leg=:topleft)
plot!(tvX_yrs, 38.08*(tvX_yrs.-yr0), label="mean( aM_tidal_JPL(t) )")
title!("Moon semimajor axis: DE430 - PlEph.jl (Float64)")
ylabel!("semimajor axis, a [millimeters]")
xlabel!(xlabel_str)
#xlims!(0,5.25)

In [ ]:
plot( tvX_yrs, 1e3diff_a_M, label="aM(DE430)-aM(PlEph.jl)", leg=:topleft)
#plot!(tvX_yrs, 38.08*(tvX_yrs.-yr0), label="mean( aM_tidal_JPL(t) )")
title!("Moon semimajor axis: DE430 - PlEph.jl (Float64)")
ylabel!("semimajor axis, a [millimeters]")
xlabel!(xlabel_str)
#xlims!(0,5.25)

In [ ]:
plot( tvX_yrs, (1e3au)*a_M_pleph, label="aM(PlEph.jl)", leg=:bottomleft)
plot!( tvX_yrs, (1e3au)*a_M_de430, label="aM(DE430)", leg=:bottomleft)
plot!(tvX_yrs, 1e3PlanetaryEphemeris.ld .+ 38.08e-3*tvX_yrs, label="ld + mean( aM_tidal(t) )")
title!("Moon semimajor axis (Float64)")
ylabel!("semimajor axis, a [m]")
xlabel!(xlabel_str)

In [ ]:
lma = 2000 #2500
#lma = 10000 
diff_a_M_ma = map(x->mean(diff_a_M[x:x+lma]), 1:(length(diff_a_M)-lma)); # diff a moving average
a_M_pleph_ma = map(x->mean(a_M_pleph[x:x+lma]), 1:(length(diff_a_M)-lma)); # a(PE) moving average
a_M_pleph_mean = mean(a_M_pleph)

In [ ]:
plot( tvX_yrs[1:length(a_M_pleph_ma)], (1e3au)*(a_M_pleph_ma.-a_M_pleph_mean), label="aM(PlEph.jl)", leg=:bottomleft)
#plot!(tvX_yrs, 1e3PlanetaryEphemeris.ld .+ 38.08e-3*tvX_yrs, label="mean( aM_tidal(t) )")
title!("Moon semimajor axis, moving average: PlEph.jl (Float64)")
ylabel!("semimajor axis, a [m]")
xlabel!(xlabel_str)

In [ ]:
plot( tvX_yrs[1:length(diff_a_M_ma)], 1e3(diff_a_M_ma.-diff_a_M_ma[1]), label="a_M", leg=:topleft)
ylabel!("semimajor axis, a [mm]")
xlabel!(xlabel_str)
xlims!(minmax(tvX_yrs[1], tvX_yrs[end])...)
title!("a_M(DE430)-a_M(PlEph.jl), moving average (Float64)")
plot!(tvX_yrs, 38.08(tvX_yrs.-yr0), label="mean( aM_tidal_JPL(t) )", leg=:topleft)
#ylims!(-5,35)

In [ ]:
plot( tvX_yrs[1:length(diff_a_M_ma)], 1e3(diff_a_M_ma.-diff_a_M_ma[1]), label="a_M")
ylabel!("semimajor axis, a [mm]")
xlabel!(xlabel_str)
xlims!(minmax(tvX_yrs[1], tvX_yrs[end])...)
title!("a_M(DE430)-a_M(PlEph.jl), moving average (Float64)")
#plot!(tvX_yrs, 38.08(tvX_yrs.-yr0), label="mean( aM_tidal_JPL(t) )")
#ylims!(-45,2)
#ylims!(-60,2)

In [ ]:
#plot(tvX_yrs, 3600*(180/pi)*(deulang[:,1]), label="Δϕ")
#plot!(tvX_yrs, 3600*(180/pi)*(deulang[:,2]), label="Δθ")
#plot!(tvX_yrs, 3600*(180/pi)*(deulang[:,3]), label="Δψ")
#xlabel!("Year")
#ylabel!("Δϕ, Δθ, Δψ [arcsec]")
#title!("Lunar physical libration Euler angles: DE430 - PlEph.jl (Float64)")

In [ ]:
((343+11)*6+6+6)

In [ ]:
1738e3*(0.3/3600)*(pi/180)

In [ ]:
plot( tvX_yrs, dxma[:,1], label="x")
plot!(tvX_yrs, dxma[:,2], label="y")
plot!(tvX_yrs, dxma[:,3], label="z")
title!("Mars heliocentric position: DE430 - PlEph.jl (Float64)")
xlabel!(xlabel_str)
ylabel!("Delta (x,y,z) [m]")
#savefig("mars_xyz.png")

In [ ]:
plot( tvX_yrs, dxce[:,1], label="x")
plot!(tvX_yrs, dxce[:,2], label="y")
plot!(tvX_yrs, dxce[:,3], label="z")
title!("Ceres heliocentric position: DE430 - PlEph.jl (Float64)")
xlabel!(xlabel_str)
ylabel!("Delta (x,y,z) [m]")
#savefig("ceres_xyz.png")

In [ ]:
plot( tvX_yrs, dxsy[:,1], label="x")
plot!(tvX_yrs, dxsy[:,2], label="y")
plot!(tvX_yrs, dxsy[:,3], label="z")
title!("Sylvia heliocentric position: DE430 - PlEph.jl (Float64)")
xlabel!(xlabel_str)
ylabel!("Delta (x,y,z) [m]")
#savefig("ceres_xyz.png")

In [ ]:
plot( tvX_yrs, dxju[:,1], label="x")
plot!(tvX_yrs, dxju[:,2], label="y")
plot!(tvX_yrs, dxju[:,3], label="z")
title!("Jupiter heliocentric position: DE430 - PlEph.jl (Float64)")
xlabel!(xlabel_str)
ylabel!("Delta (x,y,z) [m]")
#savefig("jupiter_xyz.png")

In [ ]:
plot( tvX_yrs, dxsa[:,1], label="x")
plot!(tvX_yrs, dxsa[:,2], label="y")
plot!(tvX_yrs, dxsa[:,3], label="z")
title!("Saturn heliocentric position: DE430 - PlEph.jl (Float64)")
xlabel!(xlabel_str)
ylabel!("Delta (x,y,z) [m]")

In [ ]:
plot( tvX_yrs, dxne[:,1], label="x")
plot!(tvX_yrs, dxne[:,2], label="y")
plot!(tvX_yrs, dxne[:,3], label="z")
title!("Neptune heliocentric position: DE430 - PlEph.jl (Float64)")
xlabel!(xlabel_str)
ylabel!("Delta (x,y,z) [m]")

In [ ]:
plot( tvX_yrs, dxpl[:,1], label="x")
plot!(tvX_yrs, dxpl[:,2], label="y")
plot!(tvX_yrs, dxpl[:,3], label="z")
title!("Pluto heliocentric position: DE430 - PlEph.jl (Float64)")
xlabel!(xlabel_str)
ylabel!("Delta (x,y,z) [m]")

# Position of the center of mass/energy ($N$-body metric)

In [ ]:
#using JLD

In [ ]:
sseph_ssb = "sseph_p31y_1969Jun28.jld"
#sseph_ssb = "sseph_p30y_J2000.jld"

@time t_sseph = load(sseph_ssb, "t");
@time x_sseph = load(sseph_ssb, "x");

In [ ]:
t_sseph_sub = t_sseph[1:20:end]
ind_t_sseph_sub = findall(x->x ∈ t_sseph_sub, t_sseph)

x_ssb = Array{Float64}(undef, length(ind_t_sseph_sub), 3);
for i in eachindex(ind_t_sseph_sub)
    ssb_i = ssb_posvel_pN(μ, x_sseph[ind_t_sseph_sub[i],:])
    x_ssb[i,:] = ssb_i[1]/ssb_i[3]
end

In [ ]:
t_sseph_yrs_sub = yr0 .+ (t_sseph_sub/yr)

plot(t_sseph_yrs_sub, 1e6au*x_ssb[:,1], label="x")
plot!(t_sseph_yrs_sub, 1e6au*x_ssb[:,2], label="y")
plot!(t_sseph_yrs_sub, 1e6au*x_ssb[:,3], label="z")
xlabel!("Year")
ylabel!("Position [mm]")
title!("Center of mass/energy (n-body metric)")

In [ ]:
t_sseph_yrs_sub = yr0 .+ (t_sseph_sub/yr)

plot(t_sseph_yrs_sub, 1e6au*x_ssb[:,1], label="x")
plot!(t_sseph_yrs_sub, 1e6au*x_ssb[:,2], label="y")
plot!(t_sseph_yrs_sub, 1e6au*x_ssb[:,3], label="z")
xlabel!("Year")
ylabel!("Position [mm]")
title!("Center of mass/energy (n-body metric)")

In [ ]:
t_sseph_yrs_sub = yr0 .+ (t_sseph_sub/yr)

plot(t_sseph_yrs_sub, 1e6au*x_ssb[:,1], label="x")
plot!(t_sseph_yrs_sub, 1e6au*x_ssb[:,2], label="y")
plot!(t_sseph_yrs_sub, 1e6au*x_ssb[:,3], label="z")
xlabel!("Year")
ylabel!("Position [mm]")
title!("Center of mass/energy (n-body metric)")

In [ ]:
t_sseph_yrs_sub = yr0 .+ (t_sseph_sub/yr)

plot(t_sseph_yrs_sub, 1e6au*x_ssb[:,1], label="x")
plot!(t_sseph_yrs_sub, 1e6au*x_ssb[:,2], label="y")
plot!(t_sseph_yrs_sub, 1e6au*x_ssb[:,3], label="z")
xlabel!("Year")
ylabel!("Position [mm]")
title!("Center of mass/energy (n-body metric)")

# Paper plots

In [ ]:
@time using Gaston

In [ ]:
p_ce = plot( tvX_yrs, dxce[:,1],
    Axes(
        linetype = :tab10,
        grid=true,
        title="'Ceres heliocentric position'",
        xlabel="'Year'",
        ylabel="'\$\\Delta x, \\Delta y, \\Delta z\$ [m]'",
        ytics  = "('\$-2\\cdot 10^{5}\$' -2e5, '\$- 10^{5}\$' -1e5, '0' 0, '\$10^{5}\$' 1e5, '\$2\\cdot 10^{5}\$' 2e5)"
    ),
    handle = 1
)
plot!( tvX_yrs, dxce[:,2])
plot!( tvX_yrs, dxce[:,3])

p_ea = plot( tvX_yrs, dxea[:,1],
    Axes(
        linetype = :tab10,
        grid=true,
        title="'Earth heliocentric position'",
        xlabel="'Year'",
        ylabel="'\$\\Delta x, \\Delta y, \\Delta z\$ [m]'"
    ),
    handle = 2
)
plot!( tvX_yrs, dxea[:,2])
plot!( tvX_yrs, dxea[:,3])

p_mo = plot( tvX_yrs, (drem_de430-drem_pleph),
    Axes(
        linetype = :tab10,
        grid=true,
        title="'Moon geocentric range'",
        xlabel="'Year'",
        ylabel="'\$\\Delta r\$ [m]'"
    ),
    handle = 3
)

p_ju = plot( tvX_yrs, dxju[:,1],
    Axes(
        linetype = :tab10,
        grid=true,
        title="'Jupiter heliocentric position'",
        xlabel="'Year'",
        ylabel="'\$\\Delta x, \\Delta y, \\Delta z\$ [m]'"
    ),
    handle = 4
)
plot!( tvX_yrs, dxju[:,2])
plot!( tvX_yrs, dxju[:,3])


plot([
    p_ea p_mo;
    p_ju p_ce
])

Gaston.save(term = "cairolatex", output = "pleph_gaston.tex", saveopts = "pdf standalone color dashed transparent size 8in,6in font ',12'")
run(`pdflatex pleph_gaston.tex`)

# DDA plots

In [ ]:
p_me = plot( tvX_yrs, dxme[:,1], label="x")
plot!(tvX_yrs, dxme[:,2], label="y")
plot!(tvX_yrs, dxme[:,3], label="z", leg=false)
title!("Mercury heliocentric position", titlefont = 12, guidefont=10) #: DE430 - TaylorIntegration.jl")
xlabel!(xlabel_str)
ylabel!("Δx, Δy, Δz [m]")
#yticks!(-0.04:0.02:0.04)

In [ ]:
p_ea = plot( tvX_yrs, dxea[:,1], label="x")
plot!(tvX_yrs, dxea[:,2], label="y")
plot!(tvX_yrs, dxea[:,3], label="z", leg=false)
title!("Earth heliocentric position", titlefont = 12, guidefont=10) #: DE430 - TaylorIntegration.jl")
xlabel!(xlabel_str)
ylabel!("Δx, Δy, Δz [m]")

In [ ]:
p_mo = plot(tvX_yrs, (drem_de430-drem_pleph), leg=false)
title!("Moon geocentric range", titlefont = 12, guidefont=10) #: DE430 - TaylorIntegration.jl")
ylabel!("Δr [m]")
xlabel!(xlabel_str)

In [ ]:
p_ju = plot( tvX_yrs, dxju[:,1], label="x")
plot!(tvX_yrs, dxju[:,2], label="y")
plot!(tvX_yrs, dxju[:,3], label="z", leg=false)
title!("Jupiter heliocentric position", titlefont = 12, guidefont=10) #: DE430 - TaylorIntegration.jl")
xlabel!(xlabel_str)
ylabel!("Δx, Δy, Δz [m]")

In [ ]:
plot(p_me, p_ea, p_mo, p_ju, layout=4)
#savefig("/Users/Jorge/Downloads/pleph.pdf")

In [ ]:
plot(p_me, p_ea, p_mo, p_ju, layout=4)
#savefig("/Users/Jorge/Downloads/pleph.pdf")

In [ ]:
plot(p_me, p_ea, p_mo, p_ju, layout=4)
#savefig("/Users/Jorge/Downloads/pleph.pdf")

In [ ]:
plot(p_me, p_ea, p_mo, p_ju, layout=4)
#savefig("/Users/Jorge/Downloads/pleph.pdf")

In [ ]:
plot(p_me, p_ea, p_mo, p_ju, layout=4)
#savefig("/Users/Jorge/Downloads/pleph.pdf")

# Earth-Moon barycenter and relative coordinates

In [ ]:
_pv = spkgeo(399, et0, "J2000", 0)[1]; pvE = _pv/au; pvE[4:6]=pvE[4:6]*daysec; pvE
_pv = spkgeo(301, et0, "J2000", 0)[1]; pvM = _pv/au; pvM[4:6]=pvM[4:6]*daysec; pvM;

In [ ]:
pvE = [1.0014019963481986, 0.023596413386838894, 0.01018195925304404, -0.000639155607893068, 0.015722602165727708, 0.0068170176732665335]
;

In [ ]:
pvM = [1.0003674329783419, 0.025641175831289952, 0.011164916510692924, -0.0011956034264746331, 0.015524326306986554, 0.006676849606816583]
;

In [ ]:
_pv = spkgeo(3, et0, "J2000", 0)[1]; pvEMB = _pv/au; pvEMB[4:6]=pvEMB[4:6]*daysec; pvEMB
_pv = spkgeo(301, et0, "J2000", 399)[1]; pvME = _pv/au; pvME[4:6]=pvME[4:6]*daysec; pvME;

In [ ]:
pvEMB

In [ ]:
pvME

In [ ]:
((pvEMB .- μ[mo]pvME/(μ[ea]+μ[mo])) .- pvE)./pvE

In [ ]:
((pvEMB .+ μ[ea]pvME/(μ[ea]+μ[mo])) .- pvM)./pvM

# TODO: Compute eclipses from ephemeris